This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [1]:
# get api key from your .env file
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

#print(API_KEY)

Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [3]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = f"https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X.json?start_date=2017-01-01&end_date=2017-12-31&api_key={API_KEY}"
r = requests.get(url)
json_data = r.json()

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(json_data['dataset'].keys())
print(json_data['dataset']['column_names'])
print(json_data['dataset']['data'][0])

dict_keys(['id', 'dataset_code', 'database_code', 'name', 'description', 'refreshed_at', 'newest_available_date', 'oldest_available_date', 'column_names', 'frequency', 'type', 'premium', 'limit', 'transform', 'column_index', 'start_date', 'end_date', 'data', 'collapse', 'order', 'database_id'])
['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']
['2017-12-29', 51.76, 51.94, 51.45, 51.76, None, 34640.0, 1792304.0, None, None, None]


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [5]:
#for i in json_data['dataset']['data']:
#    print(i[0])
#Missing some dates: Because markets are not open on weekends
open_list = []
delta_high_low = []
high = []
low = []
trad_vol = []
close_list = []
for var in json_data['dataset']['data']:
    open_list.append(var[1])
    delta_high_low.append(var[2]-var[3])
    high.append(var[2])
    low.append(var[3])
    trad_vol.append(var[6])
    close_list.append(var[4])

In [6]:
### Calculate what the highest and lowest opening prices were for the stock in this period.
# Using list comprehension to exclude NA data
open_list = [x for x in open_list if x is not None]
#print('\n',open_list)
high_open_price = max(open_list)
low_open_price = min(open_list)

### What was the largest change in any one day (based on High and Low price)?
max_delta_one_day = max(delta_high_low)


### Find max between two days, and two consecutive days:
max_delta_two_days_close = max(close_list)-min(close_list)


### Find average trading volume for the year 2017:
avg_trad_vol = sum(trad_vol)/len(trad_vol)

### What was the median trading volume for the year 2017:
def median(lst):
    sort_lst = sorted(lst)
    length_lst = len(lst)
    index = (length_lst - 1) // 2
   
    if (length_lst % 2):
        return sort_lst[index]
    else:
        return (sort_lst[index] + sort_lst[index + 1])/2.0

med_trad_vol = median(trad_vol)

In [7]:
print('\nHighest Open Price:', high_open_price,'\nLowest Open Price:', low_open_price,
      '\nLargest Price Delta in One Day:', max_delta_one_day,
      '\nLargest Close Price Delta:', max_delta_two_days_close,
      '\nAverage Trading Volume in 2017:', avg_trad_vol,
     '\nMedian Trading Volume in 2017:',med_trad_vol)

# Test using list comprehension to ensure the change in high/low values for each day have been calculated properly 
# in the for loop above 
#delta_high_low = [high_i - low_i for high_i, low_i in zip(high,low)]
#print(high,'\n\n',low,'\n\n',delta_high_low, '\n\n', max(delta_high_low))


Highest Open Price: 53.11 
Lowest Open Price: 34.0 
Largest Price Delta in One Day: 2.8100000000000023 
Largest Close Price Delta: 19.03 
Average Trading Volume in 2017: 89124.33725490196 
Median Trading Volume in 2017: 76286.0
